<h1 align="center">Devoir #3 : Équation de Schrödinger</h1>
<h4 align="center">Théo Dionne et Jérôme Leblanc</h4>
<h5 align="center">23 mars 2023</h5>

In [ ]:
import OHQ_wrapper as OHQw # Implémentation MT + MEF ainsi que fonctions graphiques
import matplotlib.pyplot as plt

# Paramètres graphiques
params = {
    "xtick.top": True, 
    "ytick.right": True, 
    "xtick.direction": "in", 
    "ytick.direction": "in", 
    "figure.dpi": 200
    }
plt.rcParams.update(params)

# Méthode du tir (MT)

D'abord, on cherche à résoudre l'équation de Schrödinger avec la méthode du tir. Le hamiltonien est de la forme d'un oscillateur harmonique, soit:
$$
    {\psi}'' = (x^2-2E)\psi
$$
Comme à l'habitude, pour résoudre cette équation du second ordre, on s'intéresse à un système *réduit* avec le changement de variables typique $\xi \equiv \psi'$:
$$
    \begin{bmatrix}{\psi}'\\{\xi}'\end{bmatrix} = \begin{bmatrix}\xi\\(x^2-2E)\psi\end{bmatrix}
$$
Maintenant, on fixe $\psi(-L)=0$ et ${\psi}'(-L) = 0.001$ pour ensuite trouver par itération les valeurs de $E$ telles que $\psi(L|E)=0$.

N.B.: Pour le cadrage des racines via l'algorithme proposé, il est important d'essayer le cadrage dans les deux sens ($x_1$ qui augmente ou qui diminue), car il n'y a aucune garantie sur les valeurs d'une fonction de part et d'autre d'une racine. En fait, la condition $f(x_1)=-f(x_0)$ peut être problématique pour cette raison. Par expérience, reculer l'intervalle semble mieux garantir des cadres dans ce problème.

In [ ]:
sol_mt = OHQw.solutions_mt(5, 200, 6) # Calcul des 6 premières sols avec la MT

OHQw.faire_graphique(sol_mt, 5, 5)

# Méthode des éléments finis (MEF)

La méthode des éléments finis consiste essentiellement à ramener une équation différentielle à un système d'équations linéaires diagonalisable en supposant que les fonctions propres sont approximativement leur projection sur la base des *fonctions tente*. Pour ce faire, on ramène l'équation différentielle sous la forme :
$$
    \frac{1}{2}(x^2 - \partial_x^2)\psi = E\psi
$$
En introduisant sa représentation dans la base des fonctions tente $\left\{\ket{u_i}\right\}$, on peut écrire l'équation sous la forme:
$$
    \mathrm{\mathbf{L}}^C\ket{\psi}=E\mathrm{\mathbf{M}}\ket{\psi}
$$
où $\mathrm{\mathbf{L}}^C$ est la représentation de $(x^2 - \partial_x^2)/2$ et $\mathrm{\mathbf{M}}$ est la *matrice de masse*. Les énergies (valeurs propres) et les états (vecteurs propres) sont trouvés via la résolution de cette équation aux valeurs propres généralisées.

In [ ]:
sol_mef = OHQw.solutions_mef(6, 500, 6) # Calcul des 6 premières sols avec la MEF

OHQw.faire_graphique(sol_mef, 5, 5)

## Conclusion

En résumé, on observe que les deux méthodes produisent des résultats très semblables dans des délais comparables, et ce, malgré le fait que les méthodes de résolution sont très différentes. De plus, les graphiques produits et les énergies calculées sont équivalentes aux figures dans l'énoncé du problème.

Par contre, une différence observée est la précision des énergies calculées. En fait, la méthode des éléments finis admet des énergies légèrement moins précises. En fait, les énergies de ce genre d'hamiltonien sont connues analytiquement: $E_n = n+\frac{1}{2}$.